In [1]:
##  Author : Sourabh S Yelluru
##  Institute : Birla Institute of Technology and Sciences, Pilani  


#####   Imports  #####

import requests,bs4,time,datetime
import re,json
from twilio.rest import Client
import twiliodata as td

#####  Setting up Twilio Client  #####

account_sid = td.Acct_Sid
auth_token = td.Auth_Token
client = Client(account_sid, auth_token)


#####    Functions   #####
def check():
    '''Checks if there is any change in webpage contents . i.e stats'''
    global resn, res, total_confirm_Ktk, total_death_Ktk, total_recover_Ktk
    flagc = 0
    flagd = 0
    flagf = 0
    Covidpgn = requests.get("https://api.covid19india.org/data.json",)
    cpsn = bs4.BeautifulSoup(Covidpgn.content)
    drn = cpsn.find('p')
    datan = []
    for label in drn:
        keyn = re.sub(r'\s+', ' ', str(label).strip())             #Removes all unwanted characters
        valuen = str(label.find_next_sibling()).strip()
        datan.append({ keyn : valuen })
    resn = json.loads(list(datan[0].keys())[0])                    #Gets stored as dictionary
    retlist = []
    if resn['statewise'][0]['confirmed'] != res['statewise'][0]['confirmed']:
        flagc = 1                                                  #Flags if change in confirmed cases
        diff = int(resn['statewise'][0]['confirmed'])-int(res['statewise'][0]['confirmed'])
        retlist.append({"India new confirms":diff})
    if resn['statewise'][0]['deaths'] != res['statewise'][0]['deaths']:
        flagd = 1                                                  #Flags if change in deaths
        diff = int(resn['statewise'][0]['deaths'])-int(res['statewise'][0]['deaths'])
        retlist.append({"India new deaths":diff})
    if resn['statewise'][0]['recovered'] != res['statewise'][0]['recovered']:
        flagf = 1                                                  #Flags if change in recovered cases
        diff = int(resn['statewise'][0]['recovered'])-int(res['statewise'][0]['recovered'])
        retlist.append({"India new recoveries":diff})
    if flagc or flagd or flagf:                                    #Only if there is change in National Stats
        ktkdictn = resn['statewise'][3]                            #Karnataka's stats is the 4th element in the list. Idk why. But it is.
        if flagc:
            if int(ktkdictn['confirmed']) != total_confirm_Ktk:
                diffs = int(ktkdictn['confirmed'])-total_confirm_Ktk
                retlist.append({"Karnataka new confirms":diffs})
        if flagd:
            if int(ktkdictn['deaths']) != total_death_Ktk:
                diffs = int(ktkdictn['deaths'])-total_death_Ktk
                retlist.append({"Karnataka new deaths":diffs})
        if flagc:
            if int(ktkdictn['recovered']) != total_recover_Ktk:
                diffs = int(ktkdictn['recovered'])-total_recover_Ktk
                retlist.append({"Karnataka new recoveries":diffs})
    if retlist:
        return retlist
    else:
        return None
    
def get_results():
    '''Gets current stats from covid19india.org'''
    global res
    global total_confirm_India, total_death_India, total_recover_India, confirm_since_yesterday_India,death_since_yesterday_India,recover_since_yesterday_India, Highest_state_India, confirm_in_highest_state,death_in_highest_state, recover_in_highest_state, total_confirm_Ktk, total_death_Ktk, total_recover_Ktk,confirm_since_yesterday_Ktk, death_since_yesterday_Ktk, recover_since_yesterday_Ktk
    Covidpg = requests.get("https://api.covid19india.org/data.json",)
    cps = bs4.BeautifulSoup(Covidpg.content)
    dr = cps.find('p')
    data = []
    for label in dr:
        key = re.sub(r'\s+', ' ', str(label).strip())
        value = str(label.find_next_sibling()).strip()
        data.append({ key: value })
    res = json.loads(list(data[0].keys())[0])                          #Gets stored as dictionary
    total_confirm_India = int(res['statewise'][0]['confirmed'])
    total_death_India = int(res['statewise'][0]['deaths'])
    total_recover_India = int(res['statewise'][0]['recovered'])
    confirm_since_yesterday_India = int(res['statewise'][0]['delta']['confirmed'])
    death_since_yesterday_India = int(res['statewise'][0]['delta']['deaths'])
    recover_since_yesterday_India = int(res['statewise'][0]['delta']['recovered'])
    if res['statewise'][1]['confirmed'] > res['statewise'][2]['confirmed']:     #Checks if KL or MH has highest cases
        Highest_state_India = res['statewise'][1]['state']
    else:
        Highest_state_India = res['statewise'][2]['state']
    confirm_in_highest_state = res['statewise'][1]['confirmed']
    death_in_highest_state = res['statewise'][1]['deaths']
    recover_in_highest_state = res['statewise'][1]['recovered']
    ktkdict = res['statewise'][3]
    total_confirm_Ktk = int(ktkdict['confirmed'])
    total_death_Ktk = int(ktkdict['deaths'])
    total_recover_Ktk = int(ktkdict['recovered'])
    confirm_since_yesterday_Ktk = int(ktkdict['delta']['confirmed'])
    death_since_yesterday_Ktk = int(ktkdict['delta']['deaths'])
    recover_since_yesterday_Ktk = int(ktkdict['delta']['recovered'])
    
def print_info():
    global total_confirm_India, total_death_India, total_recover_India, confirm_since_yesterday_India,death_since_yesterday_India,recover_since_yesterday_India, Highest_state_India, confirm_in_highest_state,death_in_highest_state, recover_in_highest_state, total_confirm_Ktk, total_death_Ktk, total_recover_Ktk,confirm_since_yesterday_Ktk, death_since_yesterday_Ktk, recover_since_yesterday_Ktk
    print("------------------------------")
    print("India")
    print("------------------------------")
    print("Confirm:",total_confirm_India)
    print("Dead:",total_death_India)
    print("Recovered",total_recover_India)
    print("Confirm today:",confirm_since_yesterday_India)
    print("Deaths today:",death_since_yesterday_India)
    print("Recovered today:",recover_since_yesterday_India)
    print("------------------------------")
    print(Highest_state_India)
    print("------------------------------")
    print("Confirmed:",confirm_in_highest_state)
    print("Deaths:",death_in_highest_state)
    print("Recovered:",recover_in_highest_state)
    print("------------------------------")
    print("Karnataka")
    print("------------------------------")
    print("Confirmed:",total_confirm_Ktk)
    print("Deaths:",total_death_Ktk)
    print("Recovered:",total_recover_Ktk)
    print("Confirm since today:",confirm_since_yesterday_Ktk)
    print("Deaths today:",death_since_yesterday_Ktk)
    print("------------------------------")

def get_body_info(x):
    '''Returns body of SMS to be sent on update of the website'''
    global total_confirm_India, total_death_India, total_recover_India, confirm_since_yesterday_India,death_since_yesterday_India,recover_since_yesterday_India, Highest_state_India, confirm_in_highest_state,death_in_highest_state, recover_in_highest_state, total_confirm_Ktk, total_death_Ktk, total_recover_Ktk,confirm_since_yesterday_Ktk, death_since_yesterday_Ktk, recover_since_yesterday_Ktk
    strx = ''
    for a in x:
        strx = strx + str(a)
    return '''                             
        Changes : {}
        ------------------------------
        India
        ------------------------------
        Confirm: {}
        Dead: {}
        Recovered: {}
        Confirm today: {}
        Deaths today: {}
        Recovered today: {}
        ------------------------------
        {}
        ------------------------------
        Confirmed: {}
        Deaths: {}
        Recovered: {}
        ------------------------------
        Karnataka
        ------------------------------
        Confirmed: {}
        Deaths: {}
        Recovered: {}
        Confirm since today: {}
        Deaths today: {}
        ------------------------------'''.format(strx,total_confirm_India,total_death_India,total_recover_India,confirm_since_yesterday_India,death_since_yesterday_India,recover_since_yesterday_India,Highest_state_India,confirm_in_highest_state,death_in_highest_state,recover_in_highest_state,total_confirm_Ktk,total_death_Ktk,total_recover_Ktk,confirm_since_yesterday_Ktk,death_since_yesterday_Ktk)
    
def print_if_changed():
    '''If any changes occur on the epidemic statistics, prints the changes and the new stats'''
    x = check()
    if x:
        print("Changes : ")
        for a in x:
            print(x)                                 #Prints all the changes that happened since the last upate
        print("New Stats - ")
        get_results()                                #Updates stats from the website
        print_info()                                 #Prints fresh Stats
        r = get_body_info(x)
        print("Sending message    {}".format(datetime.datetime.now()))
        try:
            send_message(r)
        except:
            print("Couldn't send message!")

def send_message(bodyinfo):
    '''Sends SMS to Clients. Arg - The body of the SMS to be sent.
    Can be sent to multiple clients at once'''
    message = client.messages.create(
                              body=bodyinfo,
                              from_=td.My_Number,
                              to=td.Dest_Number_1
                          )
    message = client.messages.create(
                              body=bodyinfo,
                              from_=td.My_Number,
                              to=td.Dest_Number_2
                          )
    message = client.messages.create(
                              body=bodyinfo,
                              from_=td.My_Number,
                              to=td.Dest_Number_3
                          )
    
def main():
    get_results()
    print_info()
    while True:
        time.sleep(10)                               #Checking occurs every 10 seconds
        retelems = print_if_changed()
        
main()

------------------------------
India
------------------------------
Confirm: 1053
Dead: 27
Recovered 86
Confirm today: 24
Deaths today: 0
Recovered today: 0
------------------------------
Maharashtra
------------------------------
Confirmed: 182
Deaths: 1
Recovered: 12
------------------------------
Karnataka
------------------------------
Confirmed: 81
Deaths: 3
Recovered: 5
Confirm since today: 0
Deaths today: 0
------------------------------
Changes : 
[{'India new confirms': 2}]
New Stats - 
------------------------------
India
------------------------------
Confirm: 1055
Dead: 27
Recovered 86
Confirm today: 26
Deaths today: 0
Recovered today: 0
------------------------------
Maharashtra
------------------------------
Confirmed: 182
Deaths: 1
Recovered: 12
------------------------------
Karnataka
------------------------------
Confirmed: 81
Deaths: 3
Recovered: 5
Confirm since today: 0
Deaths today: 0
------------------------------
Sending message    2020-03-29 18:10:35.452416
Chan

SSLError: HTTPSConnectionPool(host='api.covid19india.org', port=443): Max retries exceeded with url: /data.json (Caused by SSLError(SSLError("bad handshake: SysCallError(54, 'ECONNRESET')")))